In [1]:
def decimal_to_binary(num, precision='single'):
    # 整数部分と小数部分を分離
    integer_part = int(abs(num))
    fractional_part = abs(num) - integer_part
    
    # 整数部分を2進数に変換
    binary_int = ''
    if integer_part == 0:
        binary_int = '0'
    while integer_part > 0:
        binary_int = str(integer_part % 2) + binary_int
        integer_part //= 2
    
    # 小数部分を2進数に変換（精度に応じて桁数を調整）
    binary_frac = ''
    max_precision = 23 if precision == 'single' else 52
    while fractional_part > 0 and len(binary_frac) < max_precision:
        fractional_part *= 2
        bit = int(fractional_part)
        binary_frac += str(bit)
        fractional_part -= bit
    
    return binary_int + ('.' + binary_frac if binary_frac else '')

def to_ieee754(num, precision='single'):
    if precision not in ['single', 'double']:
        raise ValueError("精度は'single'か'double'を指定してください")
    
    # 形式に応じたパラメータを設定
    params = {
        'single': {'bias': 127, 'exp_bits': 8, 'mantissa_bits': 23},
        'double': {'bias': 1023, 'exp_bits': 11, 'mantissa_bits': 52}
    }[precision]
    
    # 符号ビットの決定
    sign = '1' if num < 0 else '0'
    
    # 2進数に変換
    binary = decimal_to_binary(abs(num), precision)
    
    # 正規化して指数を求める
    if '.' not in binary:
        binary += '.'
    point_pos = binary.find('.')
    first_one = binary.find('1')
    
    if first_one == -1:  # 0の場合
        exponent = 0
        mantissa = '0' * params['mantissa_bits']
    else:
        # 指数の計算
        exponent = point_pos - first_one - 1
        if first_one > point_pos:
            exponent = point_pos - first_one
            
        # バイアスの追加
        exponent += params['bias']
        
        # 仮数部の作成
        binary_normalized = binary.replace('.', '')
        mantissa = binary_normalized[first_one+1:]
        mantissa = mantissa + '0' * params['mantissa_bits']  # 必要な長さまで0で埋める
        mantissa = mantissa[:params['mantissa_bits']]
    
    # 指数部のビット表現
    exponent = bin(exponent)[2:].zfill(params['exp_bits'])
    
    return {
        'sign': sign,
        'exponent': exponent,
        'mantissa': mantissa,
        'ieee754': sign + exponent + mantissa,
        'format': precision
    }

# 使用例
number = -83625.7
for format in ['single', 'double']:
    result = to_ieee754(number, format)
    print(f"\n{format.capitalize()}精度形式:")
    print(f"10進数: {number}")
    print(f"符号部: {result['sign']}")
    print(f"指数部: {result['exponent']}")
    print(f"仮数部: {result['mantissa']}")
    print(f"IEEE 754: {result['ieee754']}")


Single精度形式:
10進数: -83625.7
符号部: 1
指数部: 10001111 (e=16)
仮数部: 01000110101010011011001
IEEE 754: 11000111101000110101010011011001

Double精度形式:
10進数: -83625.7
符号部: 1
指数部: 10000001111 (e=16)
仮数部: 0100011010101001101100110011001100110011001100110011
IEEE 754: 1100000011110100011010101001101100110011001100110011001100110011


In [15]:
# %pip install ieee754
from ieee754 import double, single

print(single(-83625.7))
print(double(-83625.7))

Note: you may need to restart the kernel to use updated packages.
1 10001111 01000110101010011011001
1 10000001111 0100011010101001101100110011001100110011001100110011
